Load the data and split into train and test set

In [1]:
from datasets import load_dataset

# Load the dataset and split into train and test
dataset = load_dataset("pubmed_qa", "pqa_labeled")["train"].train_test_split(test_size=0.3, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train size: 700, Test size: 300


2. Load Pre-trained Model and Evaluate on Test Subset

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import evaluate

# Load tokenizer and model
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Set up QA pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Quick evaluation on 200 samples:



In [3]:
import random

def map_answer_to_label(answer: str) -> str:
    answer = answer.lower()
    if any(x in answer for x in ["no", "not", "does not", "none", "negative"]):
        return "no"
    elif any(x in answer for x in ["yes", "does", "can", "reduce", "associated with", "positive"]):
        return "yes"
    else:
        return "maybe"

sampled_data = random.sample(list(test_dataset), 300)
preds, refs = [], []

for example in sampled_data:
    question, context, label = example["question"], example["context"], example["final_decision"]
    try:
        result = qa_pipeline({"question": question, "context": context})
        pred_label = map_answer_to_label(result["answer"])
    except:
        pred_label = "maybe"  
    preds.append(pred_label)
    refs.append(label)

label2id = {"yes": 0, "no": 1, "maybe": 2}
id2label = {v: k for k, v in label2id.items()}
# Convert to class IDs
preds_ids = [label2id.get(p, 2) for p in preds]  # default to 'maybe'
refs_ids = [label2id.get(r, 2) for r in refs]

acc = accuracy_metric.compute(predictions=preds_ids, references=refs_ids)["accuracy"]
f1 = f1_metric.compute(predictions=preds_ids, references=refs_ids, average="macro")["f1"]

print(f"Initial Accuracy: {acc:.2f}, Initial F1: {f1:.2f}")


Initial Accuracy: 0.12, Initial F1: 0.07


/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Evaluate using semantic similarity

In [4]:
from sentence_transformers import SentenceTransformer, util

model_Sentence = SentenceTransformer("all-MiniLM-L6-v2")

sim_scores = []
for pred, ref in zip(preds, refs):
    sim = util.cos_sim(model_Sentence.encode(pred), model_Sentence.encode(ref)).item()
    sim_scores.append(sim)

avg_sim = sum(sim_scores) / len(sim_scores)
print(f"Average Semantic Similarity: {avg_sim:.2f}")


Average Semantic Similarity: 0.49


train the model

In [5]:
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import torch


def preprocess(examples):
    return tokenizer(
        examples["question"],  # Keep this as is
        [' '.join(entry['contexts']) for entry in examples["context"]],  # Process each example's context separately
        truncation=True,
        padding="max_length",
        max_length=512
    )



tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_test = test_dataset.map(preprocess, batched=True)

# Dummy labels for QA (start/end tokens) to make it minimally trainable
def add_dummy_labels(example):
    example["start_positions"] = 0
    example["end_positions"] = 1
    return example

tokenized_train = tokenized_train.map(add_dummy_labels)
tokenized_test = tokenized_test.map(add_dummy_labels)



Map: 100%|██████████| 300/300 [00:00<00:00, 7369.37 examples/s]


In [6]:
tokenized_train

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 700
})

In [7]:
train_token_dataset = tokenized_train.remove_columns(["question", "final_decision", "context", "long_answer", "pubid"])


In [8]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    eval_strategy="no",
    logging_steps=10,
    save_steps=10000,
    save_total_limit=1,
    remove_unused_columns=False
)





In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_token_dataset,
    tokenizer=tokenizer
)



/tmp/ipykernel_27414/1445769881.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train
trainer.train()